In [ ]:
## Building a chatbot with Deep NLP

In [ ]:
#Importng the libraries

In [2]:
import numpy as np
import tensorflow as tf
import re
import time

In [ ]:
## Data Preprocessing

In [5]:
#import the dataset
lines = open('data/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('data/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [8]:
#Creating a dictionary that maps each line and its id
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]


In [10]:
# Creating a list of all of the conversations
conversations_ids = []

for conversation in conversations[:-1]:
    _conversation =  conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

In [ ]:
# Getting seprately the questions and the answers
questions = []
answers = []

for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])


In [15]:
# cleaning the texts

def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r"\'ve", "have", text)
    text = re.sub(r"\'d", "would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [17]:
# Cleaning the questions

clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

In [16]:
# Cleaning the answers

clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [20]:
# Creating a dictionary that maps each word to its number of occurences

word2count = {}
for questions in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
            
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
            

In [23]:
# Creating two dictionaries that map the questions words and the answers word to a unique integres
threshold = 20
questionswords2int = {}
word_number = 0

for word, count in word2count.items():
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1
        
        
answerswords2int = {}
word_number = 0

for word, count in word2count.items():
    if count >= threshold:
        answerswords2int[word] = word_number
        word_number += 1
        

In [25]:
# Adding the last tokens to these two dictionaries

tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']

for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1

for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1
    

In [26]:
# Creating the inverse dictionary of the answerwords2int dictionary
answerswords2word = {w_i: w for w, w_i in answerswords2int.items()}

In [27]:
# Adding the End of String token to the end of every answer
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>' 

In [ ]:
# Translating all the questions tand ansewes into integers
# and Replacing all the word that were filtered out by <OUT>

questions_to_int = []

for question in clean_questions:
    ints = []
    for word in questions.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_to_int.append(ints)
    
answers_to_int = []

for answer in clean_answers:
    ints = []
    for word in answers.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_to_int. append(ints)

In [ ]:
# Sorting questions and answers by the length of questions
sorted_clean_questions = []
sorted_clean_answers = []

for lenght in range(1, 25):
    for i in enumerate(questions_to_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_to_int[i[0]])
            sorted_clean_answers.append(answers_to_int[i[0]])
            
        